In [1]:
import pandas as pd
import numpy as np
import math 

import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd

df = pd.read_csv('./data/train_clean.csv')
conversor = pd.read_csv('./data/conversor.csv',sep=';')
conversor['Moneda'] = conversor['Moneda'].astype('category')
conversor['Date'] = pd.to_datetime(conversor['Date'], format='%d/%m/%Y')

In [2]:
#Normalizo Booleanas
df['Pricing, Delivery_Terms_Quote_Appr'] = df['Pricing, Delivery_Terms_Quote_Appr'].astype('boolean')
df['Pricing, Delivery_Terms_Approved'] = df['Pricing, Delivery_Terms_Approved'].astype('boolean')
df['Bureaucratic_Code_0_Approval'] = df['Bureaucratic_Code_0_Approval'].astype('boolean')
df['Bureaucratic_Code_0_Approved'] = df['Bureaucratic_Code_0_Approved'].astype('boolean')

df['Total_Amount_Currency'] = df['Total_Amount_Currency'].astype('category')
df['Stage'] = df['Stage'].astype('category')
df['Region'] = df['Region'].astype('category')

df['Source '] = df['Source '].astype('category')
df['Billing_Country'] = df['Billing_Country'].astype('category')
df['Account_Name'] = df['Account_Name'].astype('category')
df['Opportunity_Name'] = df['Opportunity_Name'].astype('category')
df['Account_Owner'] = df['Account_Owner'].astype('category')
df['Delivery_Terms'] = df['Delivery_Terms'].astype('category')
df['Product_Family'] = df['Product_Family'].astype('category')
df['Product_Name'] = df['Product_Name'].astype('category')
df['Brand'] = df['Brand'].astype('category')

df['Last_Modified_By'] = df['Last_Modified_By'].astype('category')
df['Opportunity_Owner'] = df['Opportunity_Owner'].astype('category')
df['Territory'] = df['Territory'].astype('category')

df['Total_Taxable_Amount_Currency'] = df['Total_Taxable_Amount_Currency'].astype('category')
df['Bureaucratic_Code'] = df['Bureaucratic_Code'].astype('category')
df['Account_Type'] = df['Account_Type'].astype('category')
df['Quote_Type'] = df['Quote_Type'].astype('category')
df['Opportunity_Type'] = df['Opportunity_Type'].astype('category')
df['Product_Type'] = df['Product_Type'].astype('category')
df['Product_Category_B'] = df['Product_Category_B'].astype('category')
df['Currency'] = df['Currency'].astype('category')
df['ASP_Currency'] = df['ASP_Currency'].astype('category')
df['Delivery_Quarter'] = df['Delivery_Quarter'].astype('category')
df['Delivery_Year'] = df['Delivery_Year'].astype('category')

#Normalizo Fechas
df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format='%m/%d/%Y')
df['Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'], format='%m/%d/%Y')
df['Quote_Expiry_Date'] = pd.to_datetime(df['Quote_Expiry_Date'], format='%m/%d/%Y')
df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], format='%m/%d/%Y')
df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], format='%m/%d/%Y')
df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format='%m/%d/%Y')
df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format='%m/%d/%Y')
#Normalizo Floats
df['ASP'] = pd.to_numeric(df['ASP'],downcast='float',errors='coerce')
df['ASP_(converted)'] = pd.to_numeric(df['ASP_(converted)'],downcast='float',errors='coerce')
df['Total_Amount'] = pd.to_numeric(df['Total_Amount'],downcast='float',errors='coerce')
df['Total_Taxable_Amount'] = pd.to_numeric(df['Total_Taxable_Amount'],downcast='float',errors='coerce')
df['ASP'] = pd.to_numeric(df['ASP'],downcast='float',errors='coerce')

df['Sales_Contract_No'] = pd.to_numeric(df['Sales_Contract_No'],downcast='integer',errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16824 entries, 0 to 16823
Data columns (total 45 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ID                                  16824 non-null  int64         
 1   Region                              16824 non-null  category      
 2   Territory                           16824 non-null  category      
 3   Pricing, Delivery_Terms_Quote_Appr  16824 non-null  boolean       
 4   Pricing, Delivery_Terms_Approved    16824 non-null  boolean       
 5   Bureaucratic_Code_0_Approval        16824 non-null  boolean       
 6   Bureaucratic_Code_0_Approved        16824 non-null  boolean       
 7   Bureaucratic_Code                   16824 non-null  category      
 8   Account_Created_Date                16824 non-null  datetime64[ns]
 9   Source                              16824 non-null  category      
 10  Billing_Country       

Me creo una columna con el valor en dolares de las montos. Para tener todo unificado en la misma moneda

In [3]:
df = df.merge(conversor, left_on=['Total_Amount_Currency','Last_Modified_Date'],right_on=['Moneda','Date'])
df['Total_Amount_USD'] = df.Total_Amount * df.valor
df.drop('Moneda',
  axis='columns', inplace=True)
df.drop('Date',
  axis='columns', inplace=True)
df.drop('valor',
  axis='columns', inplace=True)

In [4]:
Ahora unifico el valor de los totales con impuestos

SyntaxError: invalid syntax (<ipython-input-4-c1423009ec06>, line 1)

In [ ]:
df = df.merge(conversor, left_on=['Total_Taxable_Amount_Currency','Last_Modified_Date'],right_on=['Moneda','Date'])
df['Total_Taxable_Amount_USD'] = df.Total_Taxable_Amount * df.valor
df.drop('Moneda',
  axis='columns', inplace=True)
df.drop('Date',
  axis='columns', inplace=True)
df.drop('valor',
  axis='columns', inplace=True)

In [ ]:
df.columns

In [ ]:
df['Opportunity_TRF'] = df.groupby(['Opportunity_ID'])['TRF'].transform('sum')
df.drop_duplicates(subset=['Opportunity_ID'])

In [ ]:
aux = df.groupby(['Stage','Opportunity_TRF']).size().reset_index(name='cant')
aux = pd.pivot_table(aux,values='cant', index=['Opportunity_TRF'], columns=['Stage'], fill_value=0)
aux.columns = ["Lost","Won"]
aux = aux.apply(lambda x: x*100/sum(x), axis=1)
aux.plot(kind="bar", stacked=True,figsize=(30,15))


In [ ]:
aux2 = df[(df['Pricing, Delivery_Terms_Quote_Appr']==True)  ]\
    .groupby(['Pricing, Delivery_Terms_Approved','Region']).size().reset_index(name='cant')
aux2

In [ ]:
aux2 = df.groupby(['Region','Opportunity_TRF','Stage']).size().reset_index(name='cant')
aux2[aux2.cant>0]

In [ ]:
df.Total_Amount_Currency.value_counts

In [ ]:
df['aux_oportunitys'] = 1
ventas_x_vendedor = df.groupby('Account_Owner').agg({'Total_Amount_USD': np.sum, 'aux_oportunitys':np.sum}).reset_index()
ventas_x_vendedor.columns = ['Vendedor', 'Total Facturado', 'Cantididad Ventas']
ventas_x_vendedor